In [1]:
#import libraries

import cv2
import time
import imutils
import numpy as np
import urllib.request
from imutils.video import FPS
from imutils.video import VideoStream
from playsound import playsound  
import pyttsx3
from imutils import paths
url='http://192.168.1.2:8080/shot.jpg?rnd=426955'



#Initialize Objects and corresponding colors which the model can detect
labels = ["background", "aeroplane", "bicycle:it causes accidents so away from it", "bird", 
"boat","bottle which contains any liquid", "bus", "car", "cat", "chair where you sit comfortably", "cow", 
"diningtable where we place food items","dog:far away from them it bites you", "horse:far away from them it bites you", "motorbike:it causes accidents so away from it", "person", "pottedplant", 
"sheep","sofa:where you sit comfortably", "train", "monitor"]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

#Loading Caffe Model
print('[Status] Loading Model...')
nn = cv2.dnn.readNetFromCaffe('SSD_model_file.txt', 'SSD_pretrained.caffemodel')

#Initialize Video Stream
print('[Status] Starting Video Stream...')
vs = VideoStream(src=0).start()
time.sleep(2.0)
fps = FPS().start()
language = 'en'
#Loop Video Stream
while True:
    
    imgResp = urllib.request.urlopen(url)
    # Numpy to convert into a array
    imgNp = np.array(bytearray(imgResp.read()),dtype=np.uint8)
    
    # Finally decode the array to OpenCV usable format ;) 
    frame = cv2.imdecode(imgNp,-1)

    #Resize Frame to 400 pixels
    
    frame = imutils.resize(frame, width=400)
    (h, w) = frame.shape[:2]

    #Converting Frame to Blob
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 
    	0.007843, (300, 300), 127.5)

    #Passing Blob through network to detect and predict
    nn.setInput(blob)
    detections = nn.forward()


    #Loop over the detections
    for i in np.arange(0, detections.shape[2]):

	#Extracting the confidence of predictions
        confidence = detections[0, 0, i, 2]

        #Filtering out weak predictions
        if confidence > 0.5:
            
            #Extracting the index of the labels from the detection
            #Computing the (x,y) - coordinates of the bounding box        
            idx = int(detections[0, 0, i, 1])

            #Extracting bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            #Drawing the prediction and bounding box
            label = "{}:".format(labels[idx], confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY), colors[idx], 2)

            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)
            engine = pyttsx3.init()
            engine.setProperty('rate', 120)
            engine.setProperty('voice', "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0")
            engine.say(label)
            engine.runAndWait()

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFFb

    if key == ord('q'):
        break
    
    fps.update()

fps.stop()

print("[Info] Elapsed time: {:.2f}".format(fps.elapsed()))
print("[Info] Approximate FPS:  {:.2f}".format(fps.fps()))

cv2.destroyAllWindows()
vs.stop()



[Status] Loading Model...
[Status] Starting Video Stream...
[Info] Elapsed time: 56.75
[Info] Approximate FPS:  1.48
